In [1]:
# Load required libraries 

import pandas as pd
import os
from bs4 import BeautifulSoup

In [2]:
# Set path of the epinions file

filepath = os.path.join('epinions\D1.txt')

In [3]:
# Load data from file

with open(filepath) as data:
    soup = BeautifulSoup(data, 'lxml')

In [4]:
len(soup)

1

In [13]:
documents = soup.find_all('doc')

In [14]:
len(documents)

12000

In [150]:
documents[2]

<doc>\n<docid> 108 </docid>\n<product> Digital_Cameras/pr-Fuji_Finepix_MX-1200_Digital_Camera </product>\n<label> Yes\n<rating>  4.0\n<url> http://www.epinions.com/elec-review-B67-D4F9A30-38EAABA1-prod2 </url>\n<class> Camera </class>\n<text>\n            Ah, finally, A great little camera for the beginner at an easy to swallow price.\n\nThis is one of many of my Digital Camera Series that I will be reviewing this April.\nI feel I am highly qualified to review digital cameras as I have worked with them for three years now and use them on a daily basis.\n\nIf you need info on a specific camera feel free to email me, and I\x92ll help you best I can.\n\n<b> <i> Camera Details: </i> </b>\n\nMax Resolution - 1280x960\nFile Format \x96 JPEG\n5 Flash Modes \x96 Red eye, auto, on, off, slow synch\n4 Meg Smart Media Card storage (up to 79 lo-resolution images, 6 max res)\nFujinon Lens (super sharp)\nSoftware \x96 Adobe  PhotoDeluxe\n\n\n<b> <i> What I liked: </i> </b>\n\nThe simplicity of the c

In [114]:
tab = pd.DataFrame()

In [134]:
for idx, doc in enumerate(documents):
    tab.loc[idx,'DOCID'] = str(doc.docid.contents[0]).strip('\n').strip(' ')
    tab.loc[idx,'PRODUCT'] =  str(doc.product.contents[0]).strip('\n').strip(' ')
    tab.loc[idx,'LABEL'] =  str(doc.label.contents[0]).strip('\n').strip(' ')
    tab.loc[idx,'RATING'] =  str(doc.rating.contents[0]).strip('\n').strip(' ')
    tab.loc[idx,'URL'] =  str(doc.url.contents[0]).strip('\n').strip(' ')
    tab.loc[idx,'CLASS'] =  str(doc.find('class').contents[0]).strip('\n').strip(' ')
    tab.loc[idx,'TEXT'] =  doc.find('text').contents[0].strip('\n').strip(' ')

In [135]:
tab.tail()

,DOCID,PRODUCT,LABEL,RATING,URL,CLASS,TEXT
11995,36432,auto_Make/auto_Make-1994_Ford_Probe,Yes,5.0,http://www.epinions.com/content_110045859460,Auto,"I just traded-in my Ford Probe GT, auto. Coupe..."
11996,36437,auto_Make/auto_Make-1987_Dodge_Diplomat,Yes,5.0,http://www.epinions.com/content_38862753412,Auto,In my younger days .driving a cool car was a ...
11997,36442,auto_Make/auto_Make-2002_Nissan_Altima,Yes,4.0,http://www.epinions.com/content_46086590084,Auto,"I bought the 2.5S w/floor mats, in-cabin micro..."
11998,36447,auto_Make/auto_Make-2002_Nissan_Altima,Yes,5.0,http://www.epinions.com/content_63460839044,Auto,I got this car about two weeks ago. I have the...
11999,36452,auto_Make/auto_Make-2002_Nissan_Altima,No,4.0,http://www.epinions.com/content_88014163588,Auto,"The contestants:\n2003 Mazda 6s, 5-Speed Manua..."


In [136]:
set(tab.CLASS)

{'Auto', 'Camera'}

In [137]:
set(tab.RATING)

{'1.0', '2.0', '3.0', '4.0', '5.0'}

In [138]:
set(tab.LABEL)

{'No', 'Yes'}

In [149]:
tab.loc[2,'TEXT']

u'Ah, finally, A great little camera for the beginner at an easy to swallow price.\n\nThis is one of many of my Digital Camera Series that I will be reviewing this April.\nI feel I am highly qualified to review digital cameras as I have worked with them for three years now and use them on a daily basis.\n\nIf you need info on a specific camera feel free to email me, and I\x92ll help you best I can.'

In [139]:
fullpath = os.path.join('epinions\D1.csv')

In [147]:
tab.loc[:1,].to_csv(fullpath, sep=',',  index = False, header=True)

In [ ]:
auto = (tab[tab.CLASS == 'Auto'])

In [161]:
camera = (tab[tab.CLASS == 'Camera'])

In [159]:
len(auto[auto.LABEL == 'Yes'])

5105

In [160]:
len(auto[auto.LABEL == 'No'])

895

In [162]:
len(camera[camera.LABEL == 'Yes'])

5274

In [163]:
len(camera[camera.LABEL == 'No'])

726

In [133]:
for idx, doc in enumerate(documents[:5]):
#     print str(doc.docid.contents[0]).strip('\n').strip(' ')
#     print str(doc.product.contents[0]).strip('\n').strip(' ')
#     print  str(doc.label.contents[0]).strip('\n').strip(' ')
#     print  str(doc.rating.contents[0]).strip('\n').strip(' ')
#     print  str(doc.url.contents[0]).strip('\n').strip(' ')
#     print  str(doc.find('class').contents[0]).strip('\n').strip(' ')
    print  doc.find('text').contents[0].strip('\n').strip(' ')

I stumbled upon this camera after much research.  It's hard to understand why this camera is so hard to find; when every pro review site has given it very high marks.  I was considering the Minolta Dimage S404, Canon G2, Canon S40, Olympus D-40, Pentax Optio 430 before I discovered the Casio.  All of these camera's received good reviews, but only the Canon G2 was higher than the Casio.  However, the casio uses the exact same lens as the Canon G2.  The Casio is a great camera.  When you first take it out of the box, you realize that the camera is built very well.  Everything is laid out nicely.  It comes with a neck strap that is very comfortable.  I have used a 2.1 mgpxl camera for the last year and a half.  It was a good camera, but seemed to always be in the camera bag when I wanted to take a picture.  On our last trip to Disneyland, I missed a lot of good shots.  This camera has an excellent lens (all glass) and quality CCD (Sony).  Together they produce great pictures.  I have thre